In [20]:
import json, urllib.request, pprint

url = "https://api.skolverket.se/syllabus/v1/subjects/GRGRMAT01"

with urllib.request.urlopen(url) as r:
    payload = json.load(r)

subject = payload["subject"]

print("Top keys in subject:", list(subject.keys()))
print("\nversionInfo =", subject.get("versionInfo"))

# Titta särskilt på dessa – kan peka vidare till text/metadata:
for k in ["skolfsGrund", "skolfsAndring", "gradeScale"]:
    print(f"\n== {k} ==")
    pprint.pprint(subject.get(k))


Top keys in subject: ['code', 'name', 'typeOfSyllabus', 'startDate', 'modifiedDate', 'version', 'versionInfo', 'skolfsGrund', 'skolfsAndring', 'gradeScale', 'nameHeading', 'description', 'designation', 'purposeHeading', 'purpose', 'schoolTypes', 'centralCntHeading', 'centralContents', 'knowledgeReqsHeading', 'knowledgeRequirements']

versionInfo = Redigering av rubrik för betygskriterier

== skolfsGrund ==
'2010:37'

== skolfsAndring ==
'2022:8'

== gradeScale ==
'A-F'


In [25]:
from collections import deque

def walk_strings(obj, min_len=200, path=""):
    """Hitta alla strängfält längre än min_len; returnera (path, value)."""
    found = []
    q = deque([(obj, path)])
    while q:
        cur, p = q.popleft()
        if isinstance(cur, dict):
            for k, v in cur.items():
                q.append((v, f"{p}.{k}" if p else k))
        elif isinstance(cur, list):
            for i, v in enumerate(cur):
                q.append((v, f"{p}[{i}]"))
        elif isinstance(cur, str):
            if len(cur.strip()) >= min_len:
                found.append((p, cur))
    return found

long_texts = walk_strings(subject, min_len=200)
print(f"Hittade {len(long_texts)} längre textfält.")
for path, val in long_texts[:12]:
    print("\n---", path, "---\n")
    print(val[:1000], "...")  # förhandsvisning


Hittade 12 längre textfält.

--- description ---

<p>Matematiken har en flertusenårig historia med bidrag från många kulturer. Den utvecklas såväl ur praktiska behov som ur människans nyfikenhet och lust att utforska matematiken som sådan. Matematisk verksamhet är till sin art en kreativ, reflekterande och problemlösande aktivitet som är nära kopplad till den samhälleliga, sociala, naturvetenskapliga, tekniska och digitala ut­veck­lingen. Kunskaper i matematik ger människor förutsättningar att fatta väl­grun­dade beslut i vardagslivets många valsituationer och ökar möjligheterna att del­ta i samhällets beslutsprocesser.</p> ...

--- purpose ---

<p>Undervisningen i ämnet matematik ska syfta till att eleverna utvecklar kun­ska­per om matematik och matematikens användning i vardagen och inom oli­ka ämnesområden. Undervisningen ska bidra till att eleverna utvecklar in­tresse för matematik och tilltro till sin förmåga att använda matematik i olika sam­manhang. Den ska också ge eleverna möj

In [22]:
import re, html, textwrap

def html_to_text(s: str) -> str:
    s = html.unescape(s or "")
    s = re.sub(r"<\s*br\s*/?>", "\n", s, flags=re.I)
    s = re.sub(r"<[^>]+>", "", s)
    return s.strip()

# Visa första långa textträffen snyggt:
if long_texts:
    path, raw = long_texts[0]
    print("Fält:", path)
    print("\n" + textwrap.fill(html_to_text(raw), width=100))
else:
    print("Hittade inga långa textfält direkt i subject – texten kan kräva annan endpoint.")


Fält: description

Matematiken har en flertusenårig historia med bidrag från många kulturer. Den utvecklas såväl ur
praktiska behov som ur människans nyfikenhet och lust att utforska matematiken som sådan. Matematisk
verksamhet är till sin art en kreativ, reflekterande och problemlösande aktivitet som är nära
kopplad till den samhälleliga, sociala, naturvetenskapliga, tekniska och digitala ut­veck­lingen.
Kunskaper i matematik ger människor förutsättningar att fatta väl­grun­dade beslut i vardagslivets
många valsituationer och ökar möjligheterna att del­ta i samhällets beslutsprocesser.


In [18]:
import json, urllib.request, pandas as pd

url = "https://api.skolverket.se/syllabus/v1/subjects/GRGRMAT01"
with urllib.request.urlopen(url) as r:
    payload = json.load(r)

subject = payload["subject"]  # <- här ligger själva innehållet
pd.json_normalize(subject).T  # snabb översikt (transponerad)







,0
code,GRGRMAT01
name,Matematik
typeOfSyllabus,COURSE_SYLLABUS
startDate,2022-07-01
modifiedDate,2024-02-09
version,14
versionInfo,Redigering av rubrik för betygskriterier
skolfsGrund,2010:37
skolfsAndring,2022:8
gradeScale,A-F


In [19]:
vi = subject.get("versionInfo")
type(vi), (len(vi) if isinstance(vi, list) else None)
# Visa första posten om det är en lista:
first = vi[0] if isinstance(vi, list) and vi else vi
print(type(first))
print(first.keys() if isinstance(first, dict) else first)



<class 'str'>
Redigering av rubrik för betygskriterier


In [ ]:
# Gissa textkolumner
text_cols = [c for c in df.columns if any(k in c.lower() for k in
                                          ["text", "beskriv", "plan", "innehåll", "content", "body", "syllabus"])]
text_cols


In [12]:
# Visa kolumnnamn för att hitta textfältet (ex: "text", "beskrivning", "kursplan", "content", "body" etc.)
list(df.columns)

# Gissa vilka kolumner som är "långa" texter
text_cols = [c for c in df.columns if any(k in c.lower() for k in ["text", "beskriv", "plan", "innehåll", "content", "body"])]
text_cols


[]

In [ ]:
print(df.shape)     # Antal rader och kolumner
print(df.columns)   # Kolumnnamn
print(df.head())    # Visa de första raderna    